# Schema Alignment

In [1]:
!pip install pandas
!pip install pandas-profiling
!pip install numpy

## Phase 1: Dataset Manipulation

In [2]:
import pandas as pd
import numpy as np

#### Funzioni Di Manipolazione

In [3]:
def employees_range(employees):
    if (employees < 2):
        return "1"
    elif (employees >= 2 and employees <= 10):
        return "2 TO 10"
    elif (employees >= 11 and employees <= 50):
        return "11 TO 50"
    elif (employees >= 51 and employees <= 200):
        return "51 TO 200"
    elif (employees >= 201 and employees <= 500):
        return "201 TO 500"
    elif (employees >= 501 and employees <= 1000):
        return "501 TO 1,000"
    elif (employees >= 1001 and employees <= 5000):
        return "1,001 TO 5,000"
    elif (employees >= 5001 and employees <= 10000):
        return "5,001 TO 10,000"
    elif (employees > 10000):
        return "10,000+"
    elif (pd.isna(employees)):
        return pd.NA

In [4]:
def change_marketcap(marketcap):
    if 'T' in marketcap:
        marketcap = marketcap.replace('.', '')
        return marketcap.replace('T', '')
    else:
        return marketcap.replace('B', '')

In [5]:
def change_marketcap_low(marketcap):
    if 't' in marketcap:
        marketcap = marketcap.replace('.', '')
        return marketcap.replace('t', '')
    else:
        return marketcap.replace('b', '')

### Dataset Scarano Yahoo

In [6]:
scarano_yahoo = pd.read_csv("sources/aziende/Scarano-companies-finance.yahoo.com-1.csv")
scarano_yahoo.tail(500)

,Unnamed: 0,Unnamed: 0.1,ticker,name,website,headquarter,employees,industry,type,source
1805,2814,2814,CCMP,Cabot Microelectronics Corporation,https://www.cmcmaterials.com,"870 North Commons Drive Aurora, IL 60504 Unite...",2200,Semiconductor Equipment & Materials,NaN,yahoo
1806,2815,2815,ARMK,Aramark,https://www.aramark.com,"2400 Market Street Philadelphia, PA 19103 Unit...",248300,Restaurants,NaN,yahoo
1807,2816,2816,DECK,Deckers Outdoor Corporation,https://www.deckers.com,"250 Coromar Drive Goleta, CA 93117 United Stat...",3400,Footwear & Accessories,NaN,yahoo
1808,2817,2817,UNLRY,PT Unilever Indonesia Tbk,https://www.unilever.co.id,Grha Unilever Jl. BSD Boulevard Barat Green Of...,4968,Household & Personal Products,NaN,yahoo
1809,2818,2818,FOVSY,Ford Otomotiv Sanayi A.S.,https://www.fordotosan.com.tr,Akpinar Mahallesi Hasan Basri Cad. No: 2 Sanca...,13310,Auto Manufacturers,NaN,yahoo
...,...,...,...,...,...,...,...,...,...,...
2300,3334,3334,YETI,"YETI Holdings, Inc.",https://www.YETI.com,"7601 Southwest Parkway Austin, TX 78735 United...",701,Leisure,NaN,yahoo
2301,3335,3335,CWLDF,Crown Resorts Limited,https://www.crownresorts.com.au,Crown Towers Level 3 8 Whiteman Street Southba...,12489,Resorts & Casinos,NaN,yahoo
2302,3336,3336,FOXF,Fox Factory Holding Corp.,https://www.ridefox.com,"6634 Highway 53 Braselton, GA 30517 United Sta...",3870,Recreational Vehicles,NaN,yahoo
2303,3337,3337,CVAC,CureVac N.V.,https://www.curevac.com,Friedrich-Miescher-Strasse 15 TÃ¼bingen 72076 ...,455,Biotechnology,NaN,yahoo


In [7]:
scarano_yahoo.dtypes

Unnamed: 0        int64
Unnamed: 0.1      int64
ticker           object
name             object
website          object
headquarter      object
employees         int64
industry         object
type            float64
source           object
dtype: object

In [8]:
scarano_yahoo.isnull().sum()

Unnamed: 0         0
Unnamed: 0.1       0
ticker             0
name               0
website            0
headquarter        0
employees          0
industry          45
type            2305
source             0
dtype: int64

In [9]:
scarano_yahoo = scarano_yahoo.drop('Unnamed: 0', 1)
scarano_yahoo = scarano_yahoo.drop('Unnamed: 0.1', 1)
scarano_yahoo = scarano_yahoo.drop('type', 1)
scarano_yahoo = scarano_yahoo.drop('source', 1)
scarano_yahoo['employees']= scarano_yahoo.apply(lambda x: employees_range(x['employees']), axis=1)
scarano_yahoo.head(10)

,ticker,name,website,headquarter,employees,industry
0,HUN,Huntsman Corporation,http://www.huntsman.com,"10003 Woodloch Forest Drive The Woodlands, TX ...","5,001 TO 10,000",Chemicals
1,INPOY,InPost S.A.,http://www.inpost.eu,"2-4, rue Beck Luxemburg 1222 Luxembourg http:...",1,Specialty Business Services
2,BPHLY,Bank of the Philippine Islands,http://www.bpi.com.ph,Tower One Ayala North Exchange 6796 Ayala Aven...,"10,000+",Banks—Regional
3,RDFN,Redfin Corporation,http://www.redfin.com,"1099 Stewart Street Suite 600 Seattle, WA 9810...","1,001 TO 5,000",Real Estate Services
4,PHI,PLDT Inc.,http://pldt.com,Ramon Cojuangco Building Makati Avenue Corner ...,"10,000+",Telecom Services
5,STAA,STAAR Surgical Company,http://www.staar.com,"25651 Atlantic Ocean Drive Lake Forest, CA 926...","501 TO 1,000",Medical Instruments & Supplies
6,IMIAF,IMI plc,http://www.imiplc.com,Lakeside Solihull Parkway Birmingham Business ...,"5,001 TO 10,000",Specialty Industrial Machinery
7,NCLH,Norwegian Cruise Line Holdings Ltd.,http://www.nclhltd.com,"7665 Corporate Center Drive Miami, FL 33126 Un...","10,000+",Travel Services
8,UVRBF,Universal Robina Corporation,http://www.urc.com.ph,"Tera Tower 8th Floor Bridgetowne, East Rodrigu...","10,000+",Packaged Foods
9,AZPN,"Aspen Technology, Inc.",http://www.aspentech.com,"20 Crosby Drive Bedford, MA 01730 United State...","1,001 TO 5,000",Software—Application


### Dataset Scarano Investing

In [10]:
scarano_investing = pd.read_csv("sources/aziende/scarano_investing.csv")
scarano_investing.tail(500)

,Unnamed: 0,Unnamed: 0.1,ticker,name,website,headquarter,employees,industry,type,source
1559,3864,3864,GMLJ,Gemfields Group Ltd (GMLJ),www.gemfieldsgroup.com,Royal Chambers St. Julian’s Avenue PO Box 186 ...,2372,Metals & Mining,ORD,investing
1560,3865,3865,PMGRP,Premier Global Infrastructure Trust PLC (PMGRP),www.premierfunds.co.uk/premier-miton-global-re...,"High Street Eastgate Court Guildford, GU1 3DE ...",-,Capital Markets,Closed-End,investing
1561,3866,3866,FRP,Frp Advisory Group Plc (FRP),www.frpadvisory.com,"110 Cannon Street London, EC4N 6EU United Kingdom",457,Capital Markets,ORD,investing
1562,3867,3867,LWI,Lowland Investment Co (LWI),www.henderson.com/ukpi/fund/167/lowland-invest...,"201 Bishopsgate London, EC2M 3AE United Kingdom",-,Capital Markets,Closed-End,investing
1563,3868,3868,AAVC,Albion Venture Capital Trust PLC (AAVC),www.albion.capital/investor-centre/our-funds/a...,"1 Benjamin Street Farringdon London, EC1M 5QL ...",5,Capital Markets,ORD,investing
...,...,...,...,...,...,...,...,...,...,...
2054,4359,4359,ASLR,Asimilar Group PLC (ASLR),www.asimilargroup.com,"4 More London Riverside London, SE1 2AU United...",3,Capital Markets,ORD,investing
2055,4360,4360,IGP,Intercede Group (IGP),www.intercede.com,"Lutterworth Hall St. Mary's Road Lutterworth, ...",-,Software,ORD,investing
2056,4361,4361,ASPL,Aseana Properties Ltd (ASPL),www.aseanaproperties.com,"12 Castle Street Saint Helier, JE2 3RT Jersey",620,Real Estate Management & Development,ORD,investing
2057,4362,4362,IUG,Intelligent Ultrasound Group PLC (IUG),www.intelligentultrasound.com,Hodge House Floor 6A 114-116 St Mary Street Ca...,58,Health Care Technology,ORD,investing


In [11]:
scarano_investing.dtypes

Unnamed: 0       int64
Unnamed: 0.1     int64
ticker          object
name            object
website         object
headquarter     object
employees       object
industry        object
type            object
source          object
dtype: object

In [12]:
scarano_investing.isnull().sum()

Unnamed: 0      0
Unnamed: 0.1    0
ticker          0
name            0
website         0
headquarter     0
employees       0
industry        0
type            0
source          0
dtype: int64

In [13]:
scarano_investing = scarano_investing.drop('Unnamed: 0', 1)
scarano_investing = scarano_investing.drop('Unnamed: 0.1', 1)
scarano_investing = scarano_investing.drop('type', 1)
scarano_investing = scarano_investing.drop('source', 1)
scarano_investing['employees'] = scarano_investing['employees'].replace('-', np.nan)
scarano_investing['employees'] = scarano_investing['employees'].replace('NaN', np.nan)
scarano_investing['employees'] = scarano_investing['employees'].astype('float').astype('Int64')
scarano_investing['employees']= scarano_investing.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
scarano_investing.head(10)

,ticker,name,website,headquarter,employees,industry
0,VIS,Viscofan (VIS),www.viscofan.com,PolIgono Industrial Berroa Calle Berroa nr. 15...,"5,001 TO 10,000",Food Products
1,STLA,Stellantis NV (STLA),www.stellantis.com,"Singaporestraat 92-100 Lijnden, 1175 RA Nether...","10,000+",Automobiles
2,ICAs,ICA Gruppen AB (ICAs),www.icagruppen.se,"Box 4075 Solna, 169 04 Sweden","10,000+",Food & Staples Retailing
3,BNRGn,Brenntag AG (BNRGn),www.brenntag.com,"Messeallee 11 Essen, 45131 Germany","10,000+",Trading Companies & Distributors
4,7203,Toyota Motor Corp (7203),global.toyota/en,"1 Toyota-cho Toyota, 471-8571 Japan","10,000+",Automobiles
5,FGT,Finsbury Growth & Income Trust (FGT),www.finsburygt.com,"66 Buckingham Gate 5th Floor London, SW1E 6AU ...",<NA>,Capital Markets
6,BGSC,Bmo Global Smaller Companies PLC (BGSC),www.fandc.com/fandc-global-smaller-companies,6th Floor Quartermile 4 7a Nightingale Way Edi...,<NA>,Capital Markets
7,APAM,Aperam SA (APAM),www.aperam.com,"12C, rue Guillaume Kroll Luxembourg City, 1882...","5,001 TO 10,000",Metals & Mining
8,MCRO,Micro Focus International PLC (MCRO),www.microfocus.com,"The Lawn 22-30 Old Bath Road Newbury, RG14 1QN...","10,000+",Software
9,IMI,IMI PLC (IMI),www.imiplc.com,Lakeside Solihull Parkway Birmingham Business ...,"5,001 TO 10,000",Machinery


### Dataset AeA CompaniesMarket

In [14]:
aeacm = pd.read_csv("sources/aziende/aea_companiesmarket.csv")
aeacm.head()

,name,marketcap,country,change (1 year),rank,change (1 day),categories,share price
0,apple,$2.863 t,usa,43.34%,#1,-0.30%,👩‍💻 tech👩‍💻 software🇺🇸 dow jones💻 tech hardwar...,$174.56
1,microsoft,$2.500 t,usa,57.27%,#2,-0.56%,👩‍💻 software👩‍💻 tech🇺🇸 dow jones🎮 video games,$333.10
2,alphabet (google),$1.962 t,usa,66.03%,#3,-0.41%,🖥️ internet👩‍💻 tech👩‍💻 software,"$2,962"
3,saudi aramco,$1.851 t,saudi arabia,-2.52%,#4,-0.86%,⚡ energy🛢 oil&gas,$9.26
4,amazon,$1.766 t,usa,12.22%,#5,-1.13%,🛒 e-commerce🖥️ internet👩‍💻 tech🛍️ retail,"$3,483"


In [15]:
aeacm.dtypes

name               object
marketcap          object
country            object
change (1 year)    object
rank               object
change (1 day)     object
categories         object
share price        object
dtype: object

In [16]:
aeacm.isnull().sum()

name                  0
marketcap             0
country               0
change (1 year)     426
rank                  0
change (1 day)        0
categories         1553
share price           0
dtype: int64

In [17]:
aeacm = aeacm.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
aeacm.head(10)

,name,marketcap,country,change (1 year),rank,change (1 day),categories,share price
0,apple,$2.863 t,usa,43.34%,#1,-0.30%,tech software dow jones tech hardware electro...,$174.56
1,microsoft,$2.500 t,usa,57.27%,#2,-0.56%,software tech dow jones video games,$333.10
2,alphabet (google),$1.962 t,usa,66.03%,#3,-0.41%,internet tech software,"$2,962"
3,saudi aramco,$1.851 t,saudi arabia,-2.52%,#4,-0.86%,energy oil&gas,$9.26
4,amazon,$1.766 t,usa,12.22%,#5,-1.13%,e-commerce internet tech retail,"$3,483"
5,tesla,$1.008 t,usa,66.06%,#6,-6.10%,automakers tech electric vehicles bitcoin m...,"$1,004"
6,meta (facebook),$926.11 b,usa,18.67%,#7,0.94%,tech internet,$332.92
7,nvidia,$776.85 b,usa,nan,#8,2.24%,tech semiconductors electronics tech hardware,$311.74
8,berkshire hathaway,$636.44 b,usa,25.82%,#9,0.23%,investment,"$428,402"
9,tsmc,$619.73 b,taiwan,14.44%,#10,-1.89%,semiconductors tech manufacturing tech hardware,$119.50


###### Togliamo Change (1 Year), Rank e Change (1 Day). Troppo difficile da generalizzare il ranking e i change potrebbero non essere aggiornati

In [18]:
aeacm.rename(columns={'change (1 year)':'oneyear'}, inplace=True)
aeacm.rename(columns={'change (1 day)':'oneday'}, inplace=True)
aeacm = aeacm.drop('oneyear', 1)
aeacm = aeacm.drop('rank', 1)
aeacm = aeacm.drop('oneday', 1)


In [19]:
aeacm.rename(columns={'categories':'industry'}, inplace=True)
aeacm.rename(columns={'share price':'shareprice'}, inplace=True)
aeacm.rename(columns={'country':'headquarter'}, inplace=True)
aeacm['shareprice'] = aeacm['shareprice'].str.replace('$', '')
aeacm['shareprice'] = aeacm['shareprice'].replace(',','', regex=True)
aeacm['marketcap'] = aeacm['marketcap'].astype('string')
aeacm['marketcap'] = aeacm.apply(lambda x: change_marketcap_low(x['marketcap']), axis=1)
aeacm['marketcap'] = aeacm['marketcap'].str.replace('$', '')
aeacm['marketcap'] = aeacm['marketcap'].replace('N/A', pd.NA)
aeacm['marketcap'] = aeacm['marketcap'].replace('n/a', pd.NA)
aeacm['shareprice'] = aeacm['shareprice'].replace('N/A', pd.NA)
aeacm['shareprice'] = aeacm['shareprice'].replace('n/a', pd.NA)
aeacm.head()

<ipython-input-19-c9b1ee0c2c4a>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  aeacm['shareprice'] = aeacm['shareprice'].str.replace('$', '')
<ipython-input-19-c9b1ee0c2c4a>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  aeacm['marketcap'] = aeacm['marketcap'].str.replace('$', '')


,name,marketcap,headquarter,industry,shareprice
0,apple,2863,usa,tech software dow jones tech hardware electro...,174.56
1,microsoft,2500,usa,software tech dow jones video games,333.10
2,alphabet (google),1962,usa,internet tech software,2962
3,saudi aramco,1851,saudi arabia,energy oil&gas,9.26
4,amazon,1766,usa,e-commerce internet tech retail,3483


In [20]:
aeacm.tail(300)

,name,marketcap,headquarter,industry,shareprice
4668,pennantpark investment,0.46,usa,investment,6.99
4669,garrett motion,0.46,switzerland,automotive suppliers,7.17
4670,avadel pharmaceuticals,0.46,ireland,pharmaceuticals biotech,7.96
4671,vishay precision group,0.46,usa,nan,34.13
4672,gateway distriparks,0.46,india,transportation,3.72
...,...,...,...,...,...
4963,ziopharm oncology,0.27,usa,pharmaceuticals,1.29
4964,filatex india,0.27,india,nan,1.24
4965,limoneira,0.26,usa,agriculture real estate,15.26
4966,northeast bank,0.27,usa,banks financial services,33.94


### Dataset AeA ValueToday

In [21]:
aeavt = pd.read_csv("sources/aziende/aea_valuetoday.csv")
aeavt.head()

,name,rank in country (jan-2021),employee count as on date,number of employees,market value (jan-01-2021),company business,world rank (sep-01-2021),world rank (jan-01-2021),world rank (jan-2020),ceo,...,headquarters country,stock category,about company business,annual revenue in usd,annual results for year ending,market cap (sep-01-2021),business sector,stock exchange,isin code,chairman
0,apple,company rank in usa in 2021 is 1,as on 2020,147,2256.000 billion usd,technology,1,1.0,2.0,tim cook,...,usa,dow jones index (djia),"apple company designs, manufactures and market...","274,510 million usd",sep-2020,"2,510.000 billion usd",top technology companies in usa,nasdaq,NaN,NaN
1,microsoft corporation,company rank in usa in 2021 is 2,as on 2020,"156,439",1682.000 billion usd,technology,2,3.0,3.0,satya nadella,...,usa,dow jones index (djia),"microsoft develops software packages, windows ...","143,000 million usd",jun-2020,"2,269.000 billion usd",top technology companies in usa,nasdaq,us5949181045,NaN
2,alphabet,company rank in usa in 2021 is 4,as on 2020,"135,301",1185.000 billion usd,technology,3,5.0,4.0,sundar pichai,...,usa,s&p 500,alphabet inc is technology company and is pare...,"182,520 million usd",dec-2020,"1,934.000 billion usd",top technology companies in usa,nasdaq,NaN,NaN
3,saudi arabian oil company (saudi aramco),company rank in saudi arabia in 2021 is 1,as on 2020,"66,8",2051.500 billion usd,energy,4,2.0,1.0,amin h. al-nasser,...,saudi arabia,NaN,"saudi aramco is oil and gas exploration, refin...","232,740 million usd",dec-2020,"1,891.600 billion usd",top energy companies in saudi arabia,tadawul,NaN,NaN
4,amazon.com,company rank in usa in 2021 is 3,as on 2021,"1,335,000",1634.000 billion usd,ecommerce,5,4.0,5.0,andy jassy,...,usa,s&p 500,amazon.com is the world's biggest e-commerce p...,"386,060 million usd",dec-2020,"1,758.000 billion usd",top technology companies in usa,nasdaq,NaN,NaN


In [22]:
aeavt.dtypes

name                               object
rank in country (jan-2021)         object
employee count as on date          object
number of employees                object
market value (jan-01-2021)         object
company business                   object
world rank (sep-01-2021)           object
world rank (jan-01-2021)          float64
world rank (jan-2020)             float64
ceo                                object
company website                    object
market value (jan 1st 2020)        object
annual net income in usd           object
headquarters country               object
stock category                     object
about company business             object
annual revenue in usd              object
annual results for year ending     object
market cap (sep-01-2021)           object
business sector                    object
stock exchange                     object
isin code                          object
chairman                           object
dtype: object

In [23]:
toDrop = ['rank in country (jan-2021)', 'employee count as on date', 'market value (jan-01-2021)', 'world rank (sep-01-2021)',
         'world rank (jan-01-2021)', 'world rank (jan-2020)', 'stock category', 'about company business', 'business sector',
         'isin code', 'chairman', 'annual results for year ending', 'company website', 'stock exchange', 'market value (jan 1st 2020)', 'annual revenue in usd']
aeavt = aeavt.drop(toDrop, axis=1)
aeavt.head()
aeavt.rename(columns={'number of employees':'employees'}, inplace=True)
aeavt.rename(columns={'company business':'industry'}, inplace=True)
aeavt.rename(columns={'annual net income in usd':'revenue'}, inplace=True)
aeavt.rename(columns={'headquarters country':'headquarter'}, inplace=True)
aeavt.rename(columns={'market cap (sep-01-2021)':'marketcap'}, inplace=True)
aeavt['employees'] = aeavt['employees'].str.replace(r'[^0-9]+', '')
aeavt['employees'] = aeavt['employees'].replace('NaN', np.nan)
aeavt['employees'] = aeavt['employees'].astype('float').astype('Int64')
aeavt['employees']= aeavt.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
aeavt['marketcap'] = aeavt['marketcap'].str.replace(r'[^\d.]+', '')
aeavt.head()

<ipython-input-23-b348ec914151>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  aeavt['employees'] = aeavt['employees'].str.replace(r'[^0-9]+', '')
<ipython-input-23-b348ec914151>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  aeavt['marketcap'] = aeavt['marketcap'].str.replace(r'[^\d.]+', '')


,name,employees,industry,ceo,revenue,headquarter,marketcap
0,apple,51 TO 200,technology,tim cook,"57,410 million usd",usa,2510.000
1,microsoft corporation,"10,000+",technology,satya nadella,"44,300 million usd",usa,2269.000
2,alphabet,"10,000+",technology,sundar pichai,"40,260 million usd",usa,1934.000
3,saudi arabian oil company (saudi aramco),"501 TO 1,000",energy,amin h. al-nasser,"49,320 million usd",saudi arabia,1891.600
4,amazon.com,"10,000+",ecommerce,andy jassy,"21,330 million usd",usa,1758.000


### Dataset HirsutePippo ValueToday

In [24]:
hpvalue = pd.read_csv("sources/aziende/hp_aziende_2.csv")
hpvalue.head(25)

,Name,World Rank (Sep-01-2021),Annual Revenue in USD,Annual Net Income in USD,Market Value (Jan-01-2021),Headquarters Country,Company Business,Number of Employees,Stock Category,CEO,Stock Exchange,Company Website
0,APPLE,1,"274,510 Million USD","57,410 Million USD",2256.000 Billion USD,USA,"Mobiles & Accessories, Electronics, Technology","147,000","S&P 500, Nasdaq 100, Dow Jones Index (DJIA)",Tim Cook,Nasdaq,https://www.apple.com/
1,MICROSOFT CORPORATION,2,"143,000 Million USD","44,300 Million USD",1682.000 Billion USD,USA,"Software and IT, Laptops, Technology","156,439","S&P 500, Nasdaq 100, Dow Jones Index (DJIA)",Satya Nadella,Nasdaq,https://www.apple.com/
2,ALPHABET,3,"182,520 Million USD","40,260 Million USD",1185.000 Billion USD,USA,"Internet or Mobile App Based Business, Softwar...","135,301","Nasdaq 100, S&P 500",Sundar Pichai,Nasdaq,https://www.apple.com/
3,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),4,"232,740 Million USD","49,320 Million USD",2051.500 Billion USD,Saudi Arabia,"Oil and Gas, Chemicals, Energy","66,800",NaN,Amin H. Al-Nasser,Tadawul,https://www.apple.com/
4,AMAZON.COM,5,"386,060 Million USD","21,330 Million USD",1634.000 Billion USD,USA,"Internet or Mobile App Based Business, Cloud S...","1,335,000","Nasdaq 100, S&P 500",Andy Jassy,Nasdaq,https://www.apple.com/
5,FACEBOOK,6,"85,960 Million USD","29,150 Million USD",778.040 Billion USD,USA,"Internet or Mobile App Based Business, Social ...","58,604","Nasdaq 100, S&P 500",Mark Zuckerberg,Nasdaq,https://www.apple.com/
6,TESLA,7,"31,530 Million USD",-690 Million USD,668.900 Billion USD,USA,"Cars, Automobile, Electric Vehicles","70,757","S&P 500, Nasdaq 100",Elon Musk,"Bovespa Brazil, Nasdaq",https://www.apple.com/
7,BERKSHIRE HATHAWAY,8,"286,260 Million USD","42,521 Million USD",543.680 Billion USD,USA,"Investments, Insurance, Financial Services","391,500",S&P 500,Warren Buffett,NYSE,https://www.apple.com/
8,TAIWAN SEMICONDUCTOR MANUFACTURING COMPANY,9,"48,211 Million USD","18,670 Million USD",488.130 Billion USD,Taiwan,"Semiconductors, Electronics, Technology","51,297",Taiwan ADRs,C. C. Wei,"NYSE, Taiwan",https://www.apple.com/
9,TENCENT,10,"56,592 Million USD","14,430 Million USD",697.260 Billion USD,China,"Investments, Internet or Mobile App Based Busi...","85,858",NaN,Ma Huateng,HKeX,https://www.apple.com/


In [25]:
hpvalue.dtypes

Name                          object
World Rank (Sep-01-2021)      object
Annual Revenue in USD         object
Annual Net Income in USD      object
Market Value (Jan-01-2021)    object
Headquarters Country          object
Company Business              object
Number of Employees           object
Stock Category                object
CEO                           object
Stock Exchange                object
Company Website               object
dtype: object

In [26]:
hpvalue.isnull().sum()

Name                             0
World Rank (Sep-01-2021)         0
Annual Revenue in USD         2344
Annual Net Income in USD      2512
Market Value (Jan-01-2021)    1705
Headquarters Country             8
Company Business               188
Number of Employees           5522
Stock Category                8352
CEO                           9498
Stock Exchange                 842
Company Website                  0
dtype: int64

In [27]:
toDrop = ['World Rank (Sep-01-2021)', 'Annual Revenue in USD', 'Market Value (Jan-01-2021)', 'Stock Category',
         'Company Website', 'Stock Exchange']
hpvalue = hpvalue.drop(toDrop, axis=1)
hpvalue.head()

,Name,Annual Net Income in USD,Headquarters Country,Company Business,Number of Employees,CEO
0,APPLE,"57,410 Million USD",USA,"Mobiles & Accessories, Electronics, Technology","147,000",Tim Cook
1,MICROSOFT CORPORATION,"44,300 Million USD",USA,"Software and IT, Laptops, Technology","156,439",Satya Nadella
2,ALPHABET,"40,260 Million USD",USA,"Internet or Mobile App Based Business, Softwar...","135,301",Sundar Pichai
3,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),"49,320 Million USD",Saudi Arabia,"Oil and Gas, Chemicals, Energy","66,800",Amin H. Al-Nasser
4,AMAZON.COM,"21,330 Million USD",USA,"Internet or Mobile App Based Business, Cloud S...","1,335,000",Andy Jassy


In [28]:
hpvalue.rename(columns={'Name':'name'}, inplace=True)
hpvalue.rename(columns={'Annual Net Income in USD':'revenue'}, inplace=True)
hpvalue.rename(columns={'Company Business':'industry'}, inplace=True)
hpvalue.rename(columns={'Headquarters Country':'headquarter'}, inplace=True)
hpvalue.rename(columns={'Number of Employees':'employees'}, inplace=True)
hpvalue.rename(columns={'CEO':'ceo'}, inplace=True)
hpvalue['employees'] = hpvalue['employees'].str.replace(r'[^0-9]+', '')
hpvalue['employees'] = hpvalue['employees'].replace('NaN', np.nan)
hpvalue['employees'] = hpvalue['employees'].astype('float').astype('Int64')
hpvalue['employees']= hpvalue.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
hpvalue['revenue'] = hpvalue['revenue'].str.replace(r',', '.')
hpvalue['revenue'] = hpvalue['revenue'].str.replace(r'[^\d.]+', '')
hpvalue.head()

<ipython-input-28-69a173318080>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  hpvalue['employees'] = hpvalue['employees'].str.replace(r'[^0-9]+', '')
<ipython-input-28-69a173318080>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  hpvalue['revenue'] = hpvalue['revenue'].str.replace(r'[^\d.]+', '')


,name,revenue,headquarter,industry,employees,ceo
0,APPLE,57.410,USA,"Mobiles & Accessories, Electronics, Technology","10,000+",Tim Cook
1,MICROSOFT CORPORATION,44.300,USA,"Software and IT, Laptops, Technology","10,000+",Satya Nadella
2,ALPHABET,40.260,USA,"Internet or Mobile App Based Business, Softwar...","10,000+",Sundar Pichai
3,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),49.320,Saudi Arabia,"Oil and Gas, Chemicals, Energy","10,000+",Amin H. Al-Nasser
4,AMAZON.COM,21.330,USA,"Internet or Mobile App Based Business, Cloud S...","10,000+",Andy Jassy


### Dataset HirsutePippo Horizon

In [29]:
hphorizon = pd.read_csv("sources/aziende/hp_aziende.csv")
hphorizon.head(25)

,Name,Address,Industry,Type,Est. of Ownership,National ID,SIC Code
0,Toyota Motor Europe,"Avenue du Bourget 60, Bruxelles, 1140, BRUXELL...",Wholesale Trade,Corporation,2005.0,BE0441571714 Ondernemingsnummer,Whol autos/motor vehicles (5012)
1,Pfizer Service Company,"Hoge Wei 10, Zaventem (Brucargo), 1930, VLAAMS...",Wholesale Trade,Corporation,2002.0,BE0478242365 Ondernemingsnummer,Whol drugs/sundries (5122)
2,Janssen Pharmaceutica,"Turnhoutseweg 30, Beerse, 2340, ANTWERPEN, BEL...",Manufacturing,Corporation,2014.0,BE0403834160 Ondernemingsnummer,Mfg pharmaceutical preparations (2834)
3,Total Petrochemicals & Refining SA/NV,"Rue de l'Industrie 52, Bruxelles, 1040, BRUXEL...",Manufacturing,Corporation,2016.0,BE0403079441 Ondernemingsnummer,Petroleum refiner (2911)
4,ExxonMobil Petroleum & Chemical,"Polderdijkweg 3, Antwerpen, 2030, ANTWERPEN, B...",Manufacturing,Corporation,2017.0,BE0416375270 Ondernemingsnummer,Petroleum refiner (2911)
5,Electrabel,"Boulevard Simon Bolivar 34, Bruxelles, 1000, B...","Transportation, Communications, Electric, Gas,...",Corporation,2008.0,BE0403170701 Ondernemingsnummer,Electric services (4911)
6,Eurelec Trading,"Square de Meeûs 23, Bruxelles, 1000, BRUXELLES...",Services,Cooperative,2017.0,BE0656908546 Ondernemingsnummer,Business association (8611)
7,Ageas,"Rue du Marquis 1, Bruxelles, 1000, BRUXELLES-C...",Wholesale Trade,Corporation,2008.0,BE0451406524 Ondernemingsnummer,Whol non durable goods (5199)
8,Etablissementen Franz Colruyt,"Edingensesteenweg 196, Halle, 1500, VLAAMS BRA...",Retail Trade,Corporation,2016.0,BE0400378485 Ondernemingsnummer,Ret groceries (5411)
9,Glaxosmithkline Biologicals,"Rue de l'Institut 89, Rixensart, 1330, BRABANT...",Manufacturing,Corporation,2014.0,BE0440872918 Ondernemingsnummer,Mfg pharmaceutical preparations (2834)


In [30]:
hphorizon.dtypes

Name                  object
Address               object
Industry              object
Type                  object
Est. of Ownership    float64
National ID           object
SIC Code              object
dtype: object

In [31]:
hphorizon.isnull().sum()

Name                  0
Address               0
Industry              0
Type                  0
Est. of Ownership    33
National ID          71
SIC Code              0
dtype: int64

In [32]:
toDrop = ['Type', 'Est. of Ownership', 'National ID', 'SIC Code']
hphorizon = hphorizon.drop(toDrop, axis=1)
hphorizon.head()

,Name,Address,Industry
0,Toyota Motor Europe,"Avenue du Bourget 60, Bruxelles, 1140, BRUXELL...",Wholesale Trade
1,Pfizer Service Company,"Hoge Wei 10, Zaventem (Brucargo), 1930, VLAAMS...",Wholesale Trade
2,Janssen Pharmaceutica,"Turnhoutseweg 30, Beerse, 2340, ANTWERPEN, BEL...",Manufacturing
3,Total Petrochemicals & Refining SA/NV,"Rue de l'Industrie 52, Bruxelles, 1040, BRUXEL...",Manufacturing
4,ExxonMobil Petroleum & Chemical,"Polderdijkweg 3, Antwerpen, 2030, ANTWERPEN, B...",Manufacturing


In [33]:
hphorizon.rename(columns={'Name':'name'}, inplace=True)
hphorizon.rename(columns={'Address':'headquarter'}, inplace=True)
hphorizon.rename(columns={'Industry':'industry'}, inplace=True)
hphorizon.head()

,name,headquarter,industry
0,Toyota Motor Europe,"Avenue du Bourget 60, Bruxelles, 1140, BRUXELL...",Wholesale Trade
1,Pfizer Service Company,"Hoge Wei 10, Zaventem (Brucargo), 1930, VLAAMS...",Wholesale Trade
2,Janssen Pharmaceutica,"Turnhoutseweg 30, Beerse, 2340, ANTWERPEN, BEL...",Manufacturing
3,Total Petrochemicals & Refining SA/NV,"Rue de l'Industrie 52, Bruxelles, 1040, BRUXEL...",Manufacturing
4,ExxonMobil Petroleum & Chemical,"Polderdijkweg 3, Antwerpen, 2030, ANTWERPEN, B...",Manufacturing


### Dataset MMS FTEuropeansCompanies

In [34]:
path = 'sources/aziende/ftEuropeanCompanies2018.csv'

with open(path, 'r', encoding='utf-8') as f:
    mmsft = pd.read_csv(path, sep=';|"', engine='python').dropna(how='all', axis=1)
mmsft.tail(10)

,company_name,website,sector,country,revenue_euros,employees,founded
990,Omar Park Homes,https://omar.co.uk,Construction,United Kingdom,46M,379.0,1965
991,Pets Nature,https://petsnature.de,Ecommerce,Germany,14M,17.0,1999
992,TEMP.Service,https://tempservice.de,Support Services,Germany,3M,95.0,2012
993,Castlefield Partners,https://castlefield.com,Financial Services,United Kingdom,4M,31.0,2002
994,Urban Linker,https://urbanlinker.com,Support Services,France,2M,28.0,2009
995,VICRIS,https://vicris.it,Chemicals,Italy,4M,5.0,1989
996,FUTURECOM Austria GmbH,https://futurecom-world.com,Support Services,Austria,2M,6.0,2013
997,Digital Republic Media Group,https://digitalrepublic.com,Advertising,Germany,7M,30.0,2003
998,Jorge A. GÃ³mez Rebollo,https://n/a,Restaurants,Spain,3M,87.0,1997
999,Bioburger,https://bioburger.fr,Restaurants,France,1M,16.0,2011


In [35]:
toDrop = ['founded']
mmsft = mmsft.drop(toDrop, axis=1)

In [36]:
mmsft.rename(columns={'company_name':'name'}, inplace=True)
mmsft.rename(columns={'sector':'industry'}, inplace=True)
mmsft.rename(columns={'country':'headquarter'}, inplace=True)
mmsft.rename(columns={'revenue_euros':'revenue'}, inplace=True)
mmsft['employees'] = mmsft['employees'].replace('NaN', np.nan)
mmsft['employees'] = mmsft['employees'].astype('Int64')
mmsft['employees']= mmsft.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
mmsft['revenue'] = mmsft['revenue'].str.replace(r'M', '')
mmsft.head()

,name,website,industry,headquarter,revenue,employees
0,Deliveroo*,https://deliveroo.co.uk,Food & Beverage,United Kingdom,157,"1,001 TO 5,000"
1,Thermondo,https://thermondo.de,Energy,Germany,20,201 TO 500
2,Traventia Viajes*,https://traventia.es,Travel & Leisure,Spain,10,11 TO 50
3,Alainsa,https://alainsa.com,Construction,Spain,10,11 TO 50
4,iTravex,https://itravex.es,Travel & Leisure,Spain,22,11 TO 50


### Dataset MMS Iseg

In [37]:
path = 'sources/aziende/lsegEuropeanCompanies2018.csv'

with open(path, 'r', encoding='utf-8') as f:
    mmsiseg = pd.read_csv(path, sep=';|"|\t', engine='python',index_col = False).dropna(how='all', axis=1)
mmsiseg.head(10)

,company_name,website,sector,country,revenue_euros
0,"4EVER, SRO",www.4ever.cz,Manufacturing,Czech Republic,40M to 50M
1,77 Elektornika Kft,www.en.e77.hu,Manufacturing,Hungary,75M to 100M
2,7N A/S,www.7n.com,IT Technology,Denmark,75M to 100M
3,A. Mintikkis Farm,www.mintikkis.com,Wholesale,Cyprus,30M to 40M
4,Aarstiderne A/S,www.aarstiderne.com,Consumer Services,Denmark,75M to 100M
5,AB Åbro Bryggeri,www.abro.se,Food & Drink,Sweden,100M to 150M
6,Ableton,www.ableton.com,Technology,Germany,30M to 40M
7,Absolute,www.absoluteyachts.com,Manufacturing & Engineering,Italy,40M to 50M
8,Aceites Abril S.L.,www.aceitesabril.com,Food & Drink,Spain,150M to 250M
9,Aceitunas Cazorla,www.aceitunascazorla.com,Food & Drink,Spain,100M to 150M


In [40]:
mmsiseg.rename(columns={'company_name':'name'}, inplace=True)
mmsiseg.rename(columns={'sector':'industry'}, inplace=True)
mmsiseg.rename(columns={'country':'headquarter'}, inplace=True)
mmsiseg.rename(columns={'revenue_euros':'revenue'}, inplace=True)
mmsiseg['revenue'] = mmsiseg['revenue'].str[0:4]
mmsiseg.head()

,name,website,industry,headquarter,revenue
0,"4EVER, SRO",www.4ever.cz,Manufacturing,Czech Republic,40M
1,77 Elektornika Kft,www.en.e77.hu,Manufacturing,Hungary,75M
2,7N A/S,www.7n.com,IT Technology,Denmark,75M
3,A. Mintikkis Farm,www.mintikkis.com,Wholesale,Cyprus,30M
4,Aarstiderne A/S,www.aarstiderne.com,Consumer Services,Denmark,75M


### Zero8 Trustpilot

In [ ]:
z8trustpilot = pd.read_json('sources/aziende/Zero8-trustpilot.com-v1.json', lines=True)
z8trustpilot.head()

In [ ]:
z8trustpilot.info()

In [ ]:
z8trustpilot.dtypes

In [ ]:
z8trustpilot.isnull().sum()

In [ ]:
toDrop = ['mail', 'info', 'phone', 'reviews_count', 'overall_rating', 'excellent', 'great', 'average', 'poor', 'bad']
z8trustpilot = z8trustpilot.drop(toDrop, axis=1)

In [ ]:
z8trustpilot.rename(columns={'address':'headquarter'}, inplace=True)
z8trustpilot.head()

### Zero8 Valuetoday

In [ ]:
z8valuetoday = pd.read_json('sources/aziende/Zero8-value.today-v1.json', lines=True)
z8valuetoday.head()

In [ ]:
toDrop = ['worldRank', 'founders', 'foundedYear', 'businessSector']
z8valuetoday = z8valuetoday.drop(toDrop, axis=1)
z8valuetoday.head()

In [ ]:
z8valuetoday.rename(columns={'companyName':'name'}, inplace=True)
z8valuetoday.rename(columns={'marketValue':'marketcap'}, inplace=True)
z8valuetoday.rename(columns={'annualRevenueUSD':'revenue'}, inplace=True)
z8valuetoday.rename(columns={'headquartersCountry':'headquarter'}, inplace=True)
z8valuetoday.rename(columns={'nEmployees':'employees'}, inplace=True)
z8valuetoday.rename(columns={'CEO':'ceo'}, inplace=True)
z8valuetoday['employees'] = z8valuetoday['employees'].str.replace(r'[^0-9]+', '')
z8valuetoday['employees'] = z8valuetoday['employees'].replace('NaN', np.nan)
z8valuetoday['employees'] = z8valuetoday['employees'].astype('float').astype('Int64')
z8valuetoday['employees']= z8valuetoday.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
z8valuetoday['marketcap'] = z8valuetoday['marketcap'].str.replace(r'[^\d.]+', '')
z8valuetoday['revenue'] = z8valuetoday['revenue'].str.replace(r',', '.')
z8valuetoday['revenue'] = z8valuetoday['revenue'].str.replace(r'[^\d.]+', '')
z8valuetoday.head()

In [ ]:
z8valuetoday.dtypes

### PolloEPatatine ValueToday

In [ ]:
import json

openfile=open('sources/aziende/polloepatatine-value.today-v1.json')
jsondata=json.load(openfile)
pepvalue=pd.DataFrame(jsondata)

openfile.close()

pepvalue.head()

In [ ]:
toDrop = ['rank', 'market value']
pepvalue = pepvalue.drop(toDrop, axis=1)
pepvalue.head()

In [ ]:
pepvalue.dtypes

In [ ]:
pepvalue.rename(columns={'business':'industry'}, inplace=True)
pepvalue.rename(columns={'annual revenue':'revenue'}, inplace=True)
pepvalue.rename(columns={'country':'headquarter'}, inplace=True)
pepvalue['employees'] = pepvalue['employees'].str.replace(r'[^0-9]+', '')
pepvalue['employees'] = pepvalue['employees'].replace('', np.nan)
pepvalue['employees'] = pepvalue['employees'].replace('NaN', np.nan)
pepvalue['employees'] = pepvalue['employees'].astype('float').astype('Int64')
pepvalue['employees']= pepvalue.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
pepvalue['marketcap'] = pepvalue['marketcap'].str.replace(r'[^\d.]+', '')
pepvalue['revenue'] = pepvalue['revenue'].str.replace(r',', '.')
pepvalue['revenue'] = pepvalue['revenue'].str.replace(r'[^\d.]+', '')
pepvalue['revenue'] = pepvalue['revenue'].replace('', pd.NA)
pepvalue['website'] = pepvalue['website'].replace('', pd.NA)
pepvalue['headquarter'] = pepvalue['headquarter'].replace('', pd.NA)
pepvalue.tail(300)

In [ ]:
pepvalue.head(300)

### PolloEPatatine CompaniesMarketCap

In [ ]:
import json

openfile=open('sources/aziende/polloepatatine-companiesmarketcap.com-1.json')
jsondata=json.load(openfile)
pepmarket=pd.DataFrame(jsondata)

openfile.close()

pepmarket.head()

In [ ]:
toDrop = ['code', 'rank', 'change(1day)', 'change(1year)', 'competitor']
pepmarket = pepmarket.drop(toDrop, axis=1)
pepmarket.head()

In [ ]:
pepmarket.rename(columns={'business':'industry'}, inplace=True)
pepmarket.rename(columns={'country':'headquarter'}, inplace=True)
pepmarket.rename(columns={'sharePrice':'shareprice'}, inplace=True)
pepmarket['shareprice'] = pepmarket['shareprice'].str.replace('$', '')
pepmarket['shareprice'] = pepmarket['shareprice'].replace(',','', regex=True)
pepmarket['marketcap'] = pepmarket['marketcap'].str.replace('$', '')
pepmarket['marketcap'] = pepmarket['marketcap'].astype('string')
pepmarket['marketcap'] = pepmarket.apply(lambda x: change_marketcap(x['marketcap']), axis=1)
pepmarket['marketcap'] = pepmarket['marketcap'].replace('N/A', pd.NA)
pepmarket['shareprice'] = pepmarket['shareprice'].replace('N/A', pd.NA)
pepmarket.head()

In [ ]:
pepmarket.tail(300)

### CrispyMcData KPopFandom

In [ ]:
kpopfandom = pd.read_csv("sources/aziende/kpopfandom-companies.csv")
kpopfandom.head(3)

In [ ]:
kpopfandom.dtypes

In [ ]:
kpopfandom.isnull().sum()

In [ ]:
kpopfandom.info()

In [ ]:
toDrop = ["type", "key_people", "active_group_links","active_group_names","distributor_links","distributor_names","former_group_links","former_group_names","founded","founders","hangul","key_people_links","link","other_names","parternship_links","parternship_names","socials","soloist_links","soloist_names","subsidiary_links","subsidiary_names","websites"]
kpopfandom = kpopfandom.drop(toDrop, axis=1)
kpopfandom.head()


In [ ]:
kpopfandom.rename(columns={'locations':'headquarter'}, inplace=True)
kpopfandom.head()

### CrispyMcData KPoppingFandom

In [ ]:
kpopping = pd.read_csv("sources/aziende/kpopping-companies.csv")
kpopping.head(3)

In [ ]:
kpopping.dtypes

In [ ]:
kpopping.isnull().sum()

In [ ]:
kpopping.info()

In [ ]:
toDrop = ["link", "revenue", "active_artist_links","active_artist_names","active_artist_partnership_periods","active_group_links","active_group_names","active_group_partnership_periods","active_groups","artists_count","former_artist_links","former_artist_names","former_artist_partnership_periods","former_group_links","former_group_names","former_group_partnership_periods","former_groups","founded","founders","groups_count","introduction","native_names","parent_company","parent_company_link","subsidiary_links","subsidiary_names"]
kpopping = kpopping.drop(toDrop, axis=1)
kpopping.head()


In [ ]:
kpopping.rename(columns={'CEO':'ceo'}, inplace=True)
kpopping.rename(columns={'hometown':'headquarter'}, inplace=True)
kpopping.rename(columns={'socials':'website'}, inplace=True)
kpopping['employees'] = kpopping['employees'].astype('Int64')
kpopping['employees']= kpopping.apply(lambda x: pd.NA if pd.isna(x['employees']) else employees_range(x['employees']), axis=1)
kpopping.head()

### Querynator Indeed

In [ ]:
import json

openfile=open('sources/aziende/indeed.json')
jsondata=json.load(openfile)
indeed=pd.DataFrame(jsondata)

openfile.close()

indeed.head()

In [ ]:
toDrop = ["id", "happinessScore", "reviewScore", "jobOffersCount"]
indeed = indeed.drop(toDrop, axis=1)
indeed.head()

In [ ]:
indeed.rename(columns={'headquarters':'headquarter'}, inplace=True)
indeed.rename(columns={'employeesSize':'employees'}, inplace=True)
indeed.rename(columns={'industryType':'industry'}, inplace=True)
indeed.rename(columns={'websiteList':'website'}, inplace=True)
indeed.head()

### Querynator Yelp

In [ ]:
import json

openfile=open('sources/aziende/companies.json')
jsondata=json.load(openfile)
companies=pd.DataFrame(jsondata)

openfile.close()

companies.head()

In [ ]:
toDrop = ["id", "starsCount", "reviewsCount", "mobileNumber", "workingHours"]
companies = companies.drop(toDrop, axis=1)
companies.head()

In [ ]:
companies.rename(columns={'type':'industry'}, inplace=True)
companies.rename(columns={'address':'headquarter'}, inplace=True)
companies.head()

## Phase 2: Dataset Concatenation

In [ ]:
companies.info()

In [ ]:
indeed.info()

In [ ]:
dataset = [indeed, companies, scarano_yahoo, scarano_investing, aeavt, aeacm, hpvalue, hphorizon, mmsft, mmsiseg, z8trustpilot, z8valuetoday, pepvalue, pepmarket, kpopfandom, kpopping]

for d in dataset:
    d.info()

In [ ]:
result = pd.concat([indeed, companies, scarano_yahoo, scarano_investing, aeavt, aeacm, hpvalue, hphorizon, mmsft, mmsiseg, z8trustpilot, z8valuetoday, pepvalue, pepmarket, kpopfandom, kpopping], sort=False)
result['employees'].fillna(np.nan, inplace=True)
result['headquarter'].fillna(np.nan, inplace=True)
result['industry'].fillna(np.nan, inplace=True)
result['name'].fillna(np.nan, inplace=True)
result['website'].fillna(np.nan, inplace=True)
result['ticker'].fillna(np.nan, inplace=True)
result['ceo'].fillna(np.nan, inplace=True)
result['revenue'].fillna(np.nan, inplace=True)
result['marketcap'].fillna(np.nan, inplace=True)
result['shareprice'].fillna(np.nan, inplace=True)
result.rename(columns={'marketcap':'marketcap_M'}, inplace=True)

In [ ]:
for columns in result.columns:
    result[columns] = result[columns].str.upper() 
result.tail(10000)



In [ ]:
result.info()

In [ ]:
result.to_csv("alignedSchemas/companiesAligned.csv")

In [ ]:
from pandas_profiling import ProfileReport 

In [ ]:
#profile=ProfileReport(result)
#profile.to_notebook_iframe()